In [21]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.utils import shuffle

# Import and Concatenate Data

In [22]:
train_feature_df = pd.read_csv("Features/Train_Topological_Summary_Statistics.csv")
train_advanced_feature_df = pd.read_csv("Features/Train_Advanced_Features.csv")
train_statistical_features = pd.read_csv("Features/Train_Statistical_Features.csv")
train_wasserstein_features = pd.read_csv("Features/Train_Wasserstein_Features.csv")
train_landscape_features = pd.read_csv("Features/Train_landscape_Features.csv")

test_feature_df = pd.read_csv("Features/Test_Topological_Summary_Statistics.csv")
test_advanced_feature_df = pd.read_csv("Features/Test_Advanced_Features.csv")
test_statistical_features = pd.read_csv("Features/Test_Statistical_Features.csv")
test_wasserstein_features = pd.read_csv("Features/Test_Wasserstein_Features.csv")
test_landscape_features = pd.read_csv("Features/Test_landscape_Features.csv")


summary_statistics = pd.concat([train_feature_df, test_feature_df], ignore_index=True)
advanced_features = pd.concat([train_advanced_feature_df, test_advanced_feature_df], ignore_index=True)
statistical_features = pd.concat([train_statistical_features, test_statistical_features], ignore_index=True)
wasserstein_features =  pd.concat([train_wasserstein_features, test_wasserstein_features], ignore_index=True)
landscape_features =  pd.concat([train_landscape_features, test_landscape_features], ignore_index=True)



statistical_features = statistical_features.drop(columns = ["Label"])
wasserstein_features = wasserstein_features.drop(columns = ["Label"])
advanced_features = advanced_features.drop(columns = ["Label"])
landscape_features = landscape_features.drop(columns = ["Label"])


summary_statistics.drop(summary_statistics.columns[summary_statistics.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
statistical_features.drop(statistical_features.columns[statistical_features.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
wasserstein_features.drop(wasserstein_features.columns[wasserstein_features.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
advanced_features.drop(advanced_features.columns[advanced_features.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
landscape_features.drop(advanced_features.columns[advanced_features.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)


feature_df = summary_statistics.join(statistical_features)
feature_df = feature_df.join(wasserstein_features)
feature_df = feature_df.join(advanced_features)
#feature_df = feature_df.join(landscape_features) # makes result worse

labels = feature_df["Label"]


# Preprocess Data

In [23]:
# Split dataframe

X_train = feature_df[int(len(train_advanced_feature_df)):]
y_train = labels[int(len(train_advanced_feature_df)):]

X_test = feature_df[int(len(test_advanced_feature_df)):]
y_test = labels[int(len(test_advanced_feature_df)):]

In [24]:
rf = RandomForestClassifier(random_state=4)
rf.fit(X_train, y_train)

/Users/piabaronetzky/anaconda3/envs/time-delay-embeddings/lib/python3.10/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


RandomForestClassifier(random_state=4)

In [25]:
y_pred = rf.predict(X_test)

/Users/piabaronetzky/anaconda3/envs/time-delay-embeddings/lib/python3.10/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [26]:
accuracy = accuracy_score(y_pred, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.997779422649889


In [27]:
np.unique(y_pred, return_counts=True)

(array([1, 3, 5, 7]), array([ 253, 1050,   42,    6]))

In [28]:
np.unique(y_test, return_counts=True)

(array([1, 3, 5, 7]), array([ 250, 1050,   45,    6]))